### 1. Importing necessary libraries

In [59]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from bs4 import BeautifulSoup

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Dataset-1

In [60]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_areas_of_London")
result

<Response [200]>

In [61]:
soup = BeautifulSoup(result.content,'html.parser')
table = soup.find_all('table')

In [62]:
location=[]
london_borough=[]
post_town=[]
postcode=[]

for t in table:
    table_rows = t.find_all('tr')
    
    for row in table_rows:
        data = row.find_all('td')
        
        if len(data) > 1:
            d0 = data[0]
            location.append(d0.text.strip())
            d1 = data[1]
            london_borough.append(d1.text.strip())
        if len(data)>3:
            d2 = data[2]
            post_town.append(d2.text.strip())
            d3 = data[3]
            postcode.append(d3.text.strip())



In [63]:
postcode = postcode[:-1]
location = location[:-2]
london_borough = london_borough[:-2]
post_town = post_town[:-1]

In [64]:
df_1 = pd.DataFrame({'Neighborhood':location,'Borough': london_borough,'Post_town': post_town,'Post_code': postcode})
df_1

,Neighborhood,Borough,Post_town,Post_code
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Addington,Croydon[8],CROYDON,CR0
3,Addiscombe,Croydon[8],CROYDON,CR0
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
5,Aldborough Hatch,Redbridge[9],ILFORD,IG2
6,Aldgate,City[10],LONDON,EC3
7,Aldwych,Westminster[10],LONDON,WC2
8,Alperton,Brent[11],WEMBLEY,HA0
9,Anerley,Bromley[11],LONDON,SE20


Stripping the numbers from Borough column.

In [65]:
df_1['Borough'] = df_1['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df_1

,Neighborhood,Borough,Post_town,Post_code
0,Abbey Wood,"Bexley, Greenwich",LONDON,SE2
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
2,Addington,Croydon,CROYDON,CR0
3,Addiscombe,Croydon,CROYDON,CR0
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
5,Aldborough Hatch,Redbridge,ILFORD,IG2
6,Aldgate,City,LONDON,EC3
7,Aldwych,Westminster,LONDON,WC2
8,Alperton,Brent,WEMBLEY,HA0
9,Anerley,Bromley,LONDON,SE20


#### Dataset-2

In [67]:
url2 = 'https://en.wikipedia.org/wiki/Demography_of_London'

result2 = requests.get(url2)

In [68]:
soup2 = BeautifulSoup(result2.content,'html.parser')

In [69]:
table2 = soup2.find_all("table", {"class":"wikitable sortable"})
table2

[<table class="wikitable sortable" style="text-align:left">
 <tbody><tr>
 <th>Local authority</th>
 <th>White</th>
 <th>Mixed</th>
 <th>Asian</th>
 <th>Black</th>
 <th>Other
 </th></tr>
 <tr>
 <td>Barnet</td>
 <td>64.1</td>
 <td>4.8</td>
 <td>18.5</td>
 <td>7.7</td>
 <td>4.8
 </td></tr>
 <tr>
 <td>Barking and Dagenham</td>
 <td>58.3</td>
 <td>4.2</td>
 <td>15.9</td>
 <td>20</td>
 <td>1.6
 </td></tr>
 <tr>
 <td>Bexley</td>
 <td>81.9</td>
 <td>2.3</td>
 <td>6.6</td>
 <td>8.5</td>
 <td>0.8
 </td></tr>
 <tr>
 <td>Brent</td>
 <td>36.3</td>
 <td>5.1</td>
 <td>34.1</td>
 <td>18.8</td>
 <td>5.8
 </td></tr>
 <tr>
 <td>Bromley</td>
 <td>84.3</td>
 <td>3.5</td>
 <td>5.2</td>
 <td>6</td>
 <td>0.9
 </td></tr>
 <tr>
 <td>Camden</td>
 <td>66.3</td>
 <td>5.6</td>
 <td>16.1</td>
 <td>8.2</td>
 <td>3.8
 </td></tr>
 <tr>
 <td>City of London</td>
 <td>78.6</td>
 <td>3.9</td>
 <td>12.7</td>
 <td>2.6</td>
 <td>2.1
 </td></tr>
 <tr>
 <td>Croydon</td>
 <td>55.1</td>
 <td>6.6</td>
 <td>16.4</td>
 <td>20.2</td>

In [70]:
local_authority = []
white = []
mixed = []
asian = []
black = []
others = []

for t in table2:
    row = t.find_all('tr')
    
    for r in row:
        row_data = r.find_all('td')
        
        if len(row_data)>1:
            d0 = row_data[0]
            local_authority.append(d0.text.strip())
            d1 = row_data[1]
            white.append(d1.text.strip())
            d2 = row_data[2]
            mixed.append(d2.text.strip())
            d3 = row_data[3]
            asian.append(d3.text.strip())
            d4 = row_data[4]
            black.append(d4.text.strip())
            d5 = row_data[5]
            others.append(d5.text.strip())
    break

In [71]:
df_demographics = pd.DataFrame({'Local_authority':local_authority,'White': white,'Mixed': mixed,'Asian':asian,'Black': black,'Others': others})
df_demographics

,Local_authority,White,Mixed,Asian,Black,Others
0,Barnet,64.1,4.8,18.5,7.7,4.8
1,Barking and Dagenham,58.3,4.2,15.9,20,1.6
2,Bexley,81.9,2.3,6.6,8.5,0.8
3,Brent,36.3,5.1,34.1,18.8,5.8
4,Bromley,84.3,3.5,5.2,6,0.9
5,Camden,66.3,5.6,16.1,8.2,3.8
6,City of London,78.6,3.9,12.7,2.6,2.1
7,Croydon,55.1,6.6,16.4,20.2,1.8
8,Ealing,49,4.5,29.7,10.9,6
9,Enfield,61,5.5,11.2,17.2,5.1


To sort the dataframe, we check the datatypes of the columns and change to float if necessary.

In [72]:
df_demographics.dtypes

Local_authority    object
White              object
Mixed              object
Asian              object
Black              object
Others             object
dtype: object

In [73]:
df_demographics = df_demographics.astype({"White":float,"Mixed":float,"Asian":float,"Black":float,"Others":float})
df_demographics 

,Local_authority,White,Mixed,Asian,Black,Others
0,Barnet,64.1,4.8,18.5,7.7,4.8
1,Barking and Dagenham,58.3,4.2,15.9,20.0,1.6
2,Bexley,81.9,2.3,6.6,8.5,0.8
3,Brent,36.3,5.1,34.1,18.8,5.8
4,Bromley,84.3,3.5,5.2,6.0,0.9
5,Camden,66.3,5.6,16.1,8.2,3.8
6,City of London,78.6,3.9,12.7,2.6,2.1
7,Croydon,55.1,6.6,16.4,20.2,1.8
8,Ealing,49.0,4.5,29.7,10.9,6.0
9,Enfield,61.0,5.5,11.2,17.2,5.1


Sorting the dataframe in descending order with 'Asian' column.

In [74]:
df_demographics.sort_values(by='Asian', inplace=True, ascending=False)
df_demographics.reset_index(inplace=True)
df_demographics.drop('index', axis=1, inplace=True)
df_demographics.head()

,Local_authority,White,Mixed,Asian,Black,Others
0,Newham,29.0,4.5,43.5,19.6,3.5
1,Harrow,42.2,4.0,42.6,8.2,2.9
2,Redbridge,42.5,4.1,41.8,8.9,2.7
3,Tower Hamlets,45.2,4.1,41.1,7.3,2.3
4,Hounslow,51.4,4.1,34.4,6.6,3.6


In [75]:
top_8 = ['Newham', 'Redbridge','Tower Hamlets','Hounslow','Brent', 'Ealing','Hillingdon','Waltham Forest']

In [76]:
df_final = df_1[df_1['Borough'].isin(top_8)].reset_index(drop=True)
df_final.loc[10:10,'Neighborhood':'Neighborhood'] = 'Bromley'
df_final.head(11)

,Neighborhood,Borough,Post_town,Post_code
0,Aldborough Hatch,Redbridge,ILFORD,IG2
1,Alperton,Brent,WEMBLEY,HA0
2,Barkingside,Redbridge,ILFORD,IG6
3,Beckton,Newham,"LONDON, BARKING","E6, E16, IG11"
4,Bedford Park,Ealing,LONDON,W4
5,Bethnal Green,Tower Hamlets,LONDON,E2
6,Blackwall,Tower Hamlets,LONDON,E14
7,Bow,Tower Hamlets,LONDON,E3
8,Brentford,Hounslow,BRENTFORD,TW8
9,Brent Park,Brent,LONDON,NW10


#### Getting coordinates using geolocator

In [77]:
address = 'Alperton'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Alperton, London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Alperton, London are 51.5408036, -0.3000963.


In [78]:
n_list = df_final['Neighborhood'].tolist()
n_list[10] = 'Bromley'
n_list

['Aldborough Hatch',
 'Alperton',
 'Barkingside',
 'Beckton',
 'Bedford Park',
 'Bethnal Green',
 'Blackwall',
 'Bow',
 'Brentford',
 'Brent Park',
 'Bromley',
 'Brondesbury',
 'Cambridge Heath',
 'Canary Wharf',
 'Cann Hall',
 'Canning Town',
 'Chingford',
 'Church End',
 'Cowley',
 'Cranford',
 'Cubitt Town',
 'Custom House',
 'Dollis Hill',
 'Ealing',
 'East Bedfont',
 'East Ham',
 'Eastcote',
 'Feltham',
 'Forest Gate',
 'Gants Hill',
 'Goodmayes',
 'Greenford',
 'Grove Park',
 'Gunnersbury',
 'Hainault',
 'Hanwell',
 'Hanworth',
 'Harefield',
 'Harlesden',
 'Harlington',
 'Harmondsworth',
 'Hatton',
 'Hayes',
 'Heston',
 'Highams Park',
 'Hillingdon',
 'Hounslow',
 'Ickenham',
 'Ilford',
 'Isle of Dogs',
 'Isleworth',
 'Kensal Green',
 'Kingsbury',
 'Lampton',
 'Leamouth',
 'Leyton',
 'Leytonstone',
 'Limehouse',
 'Little Ilford',
 'Longford',
 'Loxford',
 'Manor Park',
 'Maryland',
 'Mile End',
 'Millwall',
 'Neasden',
 'Newbury Park',
 'North Woolwich',
 'Northolt',
 'Northwood'

In [79]:
lat = []
lng = []
i = 0
n = len(n_list)

while n > 0:
    
    try:
        address = str(n_list[i]) + ', London, UK'

        geolocator = Nominatim(user_agent="london_explorer")
        location = geolocator.geocode(address)

        #latitude = location.latitude
        lat.append(location.latitude)
        lng.append(location.longitude)
        #longitude = location.longitude
        print('The geograpical coordinate of {}, London are {}, {}.'.format(n_list[i],lat[i], lng[i]))
        i = i+1
        n = n-1
        
    except:
        
        lat.append(np.nan)
        lng.append(np.nan)
        print('The geograpical coordinate of {}, London are {}, {}.'.format(n_list[i],lat[i], lng[i]))
        i = i+1
        n = n-1

The geograpical coordinate of Aldborough Hatch, London are nan, nan.
The geograpical coordinate of Alperton, London are 51.5408036, -0.3000963.
The geograpical coordinate of Barkingside, London are 51.5858181, 0.0886245.
The geograpical coordinate of Beckton, London are 51.5160797, 0.0594257.
The geograpical coordinate of Bedford Park, London are 51.498020100000005, -0.25564718577152973.
The geograpical coordinate of Bethnal Green, London are 51.5303456, -0.0561633.
The geograpical coordinate of Blackwall, London are 51.5079378, -0.0071843.
The geograpical coordinate of Bow, London are 51.5283085, -0.0194816.
The geograpical coordinate of Brentford, London are 51.4863958, -0.3216623.
The geograpical coordinate of Brent Park, London are 51.563825800000004, -0.2757596561855699.
The geograpical coordinate of Bromley, London are 51.4028046, 0.0148142.
The geograpical coordinate of Brondesbury, London are 51.5450493, -0.2025961.
The geograpical coordinate of Cambridge Heath, London are 51.5

The geograpical coordinate of Willesden, London are 51.5466216, -0.2358661.
The geograpical coordinate of Woodford, London are 51.6068055, 0.0340119.
The geograpical coordinate of Woodlands, London are 51.4721654, -0.3374321.
The geograpical coordinate of Yeading, London are 51.527239, -0.3992705.
The geograpical coordinate of Yiewsley, London are 51.5128661, -0.4741522.


In [80]:
df_ll = pd.DataFrame({'Neighborhood':n_list,'Latitude':lat,'Longitude':lng})
df_ll

,Neighborhood,Latitude,Longitude
0,Aldborough Hatch,NaN,NaN
1,Alperton,51.540804,-0.300096
2,Barkingside,51.585818,0.088624
3,Beckton,51.516080,0.059426
4,Bedford Park,51.498020,-0.255647
5,Bethnal Green,51.530346,-0.056163
6,Blackwall,51.507938,-0.007184
7,Bow,51.528309,-0.019482
8,Brentford,51.486396,-0.321662
9,Brent Park,51.563826,-0.275760


In [81]:
df_ll['Latitude'].isna().value_counts()

False    111
True       1
Name: Latitude, dtype: int64

In [82]:
df_ll.loc[0:0,'Latitude':'Longitude'] = 51.585590, 0.098750
df_ll

,Neighborhood,Latitude,Longitude
0,Aldborough Hatch,51.585590,0.098750
1,Alperton,51.540804,-0.300096
2,Barkingside,51.585818,0.088624
3,Beckton,51.516080,0.059426
4,Bedford Park,51.498020,-0.255647
5,Bethnal Green,51.530346,-0.056163
6,Blackwall,51.507938,-0.007184
7,Bow,51.528309,-0.019482
8,Brentford,51.486396,-0.321662
9,Brent Park,51.563826,-0.275760


#### Merging the coordinates dataframe (df_ll) with neighborhoods dataframe ()

In [83]:
df = pd.merge(df_final,df_ll,on='Neighborhood')
df.head()

,Neighborhood,Borough,Post_town,Post_code,Latitude,Longitude
0,Aldborough Hatch,Redbridge,ILFORD,IG2,51.585590,0.098750
1,Alperton,Brent,WEMBLEY,HA0,51.540804,-0.300096
2,Barkingside,Redbridge,ILFORD,IG6,51.585818,0.088624
3,Beckton,Newham,"LONDON, BARKING","E6, E16, IG11",51.516080,0.059426
4,Bedford Park,Ealing,LONDON,W4,51.498020,-0.255647


#### Creating a London map

In [84]:
# Getting London Coordinates

address = 'London, UK'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of London are 51.5073219, -0.1276474.


In [85]:
# Creating a map
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)
map_london

#### Creating a map of London with neighborhoods superimposed on top

In [86]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

#### Foursquare API credentials:

In [87]:
CLIENT_ID = 'WXYMQPKTQLPFERYAPR5OU1AFFFZHLQYDBGIKTU1SVVSSNEZU' # your Foursquare ID
CLIENT_SECRET = 'KTUGM4LGMQZZPISTY22NR0DW5PAUK4BNFBM0XFLDHMKDQXP2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WXYMQPKTQLPFERYAPR5OU1AFFFZHLQYDBGIKTU1SVVSSNEZU
CLIENT_SECRET:KTUGM4LGMQZZPISTY22NR0DW5PAUK4BNFBM0XFLDHMKDQXP2


#### Exploring the first neighbourhood

In [88]:
df.loc[1, 'Neighborhood']

'Alperton'

In [89]:
neighborhood_latitude = df.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Alperton are 51.5408036, -0.3000963.


In [90]:
radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=WXYMQPKTQLPFERYAPR5OU1AFFFZHLQYDBGIKTU1SVVSSNEZU&client_secret=KTUGM4LGMQZZPISTY22NR0DW5PAUK4BNFBM0XFLDHMKDQXP2&ll=51.5408036,-0.3000963&v=20180605&radius=1000&limit=100'

In [91]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edbb16c1187ee001bb25b15'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Wembley Central',
  'headerFullLocation': 'Wembley Central, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 51.54980360900001,
    'lng': -0.28565285302799626},
   'sw': {'lat': 51.531803590999985, 'lng': -0.3145397469720037}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd1c71f77b29c7424318d82',
       'name': 'The Gym',
       'location': {'address': '197 Ealing Rd, The Atlip Centre',
        'lat': 51.54081869411914,
        'lng': -0.29871454849086343,
        'labeledLatLngs': [{'label': 'display

In [92]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [93]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/tarun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Gym,Gym / Fitness Center,51.540819,-0.298715
1,Sainsbury's,Supermarket,51.538740,-0.303272
2,Maru Bhajias,Indian Restaurant,51.543873,-0.297200
3,Subway,Sandwich Place,51.541707,-0.297996
4,East Pan Asian Restaurant,Asian Restaurant,51.537700,-0.301996
5,Goals Soccer Centre,Soccer Field,51.534539,-0.305176
6,Loon Fung,Supermarket,51.537559,-0.301984
7,Fawanees,Hookah Bar,51.533734,-0.295594
8,Currys PC World Featuring Carphone Warehouse,Electronics Store,51.535312,-0.298250
9,The Fox & Goose Hotel,Pub,51.533215,-0.295605


#### Exploring all the Neighbourhoods

In [94]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 'Venue Latitude', 
                  'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [96]:
london_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Aldborough Hatch
Alperton
Barkingside
Beckton
Bedford Park
Bethnal Green
Blackwall
Bow
Brentford
Brent Park
Bromley
Brondesbury
Cambridge Heath
Canary Wharf
Cann Hall
Canning Town
Chingford
Church End
Cowley
Cranford
Cubitt Town
Custom House
Dollis Hill
Ealing
East Bedfont
East Ham
Eastcote
Feltham
Forest Gate
Gants Hill
Goodmayes
Greenford
Grove Park
Gunnersbury
Hainault
Hanwell
Hanworth
Harefield
Harlesden
Harlington
Harmondsworth
Hatton
Hayes
Heston
Highams Park
Hillingdon
Hounslow
Ickenham
Ilford
Isle of Dogs
Isleworth
Kensal Green
Kingsbury
Lampton
Leamouth
Leyton
Leytonstone
Limehouse
Little Ilford
Longford
Loxford
Manor Park
Maryland
Mile End
Millwall
Neasden
Newbury Park
North Woolwich
Northolt
Northwood
Norwood Green
Old Ford
Osterley
Perivale
Plaistow
Poplar
Preston
Queen's Park
Ratcliff
Redbridge
Ruislip
Seven Kings
Shadwell
Silvertown
Sipson
South Ruislip
South Woodford
Southall
Spitalfields
Stepney
Stonebridge
Stratford
Tokyngton
Tower Hill
Upper Walthamstow
Upton Park
Uxb

In [97]:
print(london_venues.shape)
london_venues.head()

(4687, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aldborough Hatch,51.58559,0.09875,Miller & Carter,51.585350,0.102610,Steakhouse
1,Aldborough Hatch,51.58559,0.09875,Redbridge FC,51.585761,0.088673,Soccer Field
2,Aldborough Hatch,51.58559,0.09875,Barkingside London Underground Station,51.585392,0.088590,Metro Station
3,Aldborough Hatch,51.58559,0.09875,Sainsbury's,51.581451,0.087260,Supermarket
4,Aldborough Hatch,51.58559,0.09875,Jalalabad 2 Restaurant,51.577818,0.099732,Indian Restaurant


Let's check how many venues were returned for each neighborhood

In [98]:
london_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aldborough Hatch,8,8,8,8,8,8
Alperton,20,20,20,20,20,20
Barkingside,21,21,21,21,21,21
Beckton,25,25,25,25,25,25
Bedford Park,100,100,100,100,100,100
Bethnal Green,100,100,100,100,100,100
Blackwall,100,100,100,100,100,100
Bow,42,42,42,42,42,42
Brent Park,78,78,78,78,78,78


In [99]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 311 uniques categories.


In [100]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Football Field,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outlet Mall,Outlet Store,Paintball Field,Pakistani Restaurant,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Romanian Restaurant,Roof Deck,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street F

Neighborhoods with Asian restaurant can be viewed with the following command:

In [101]:
london_onehot.loc[london_onehot['Asian Restaurant'] != 0]

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Football Field,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outlet Mall,Outlet Store,Paintball Field,Pakistani Restaurant,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Romanian Restaurant,Roof Deck,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street F

In [102]:
london_onehot.shape

(4687, 312)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [103]:
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Canal Lock,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Football Field,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outlet Mall,Outlet Store,Paintball Field,Pakistani Restaurant,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Romanian Restaurant,Roof Deck,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street F

In [104]:
london_grouped.shape

(112, 312)

#### Printing each neighborhood along with the top 5 most common venues

In [105]:
num_top_venues = 5

for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Aldborough Hatch----
                 venue  freq
0          Social Club  0.12
1  Sporting Goods Shop  0.12
2          Supermarket  0.12
3           Steakhouse  0.12
4        Metro Station  0.12


----Alperton----
               venue  freq
0  Indian Restaurant  0.15
1     Clothing Store  0.10
2        Supermarket  0.10
3  Electronics Store  0.05
4               Park  0.05


----Barkingside----
                venue  freq
0         Supermarket  0.14
1   Indian Restaurant  0.10
2                Café  0.10
3    Greek Restaurant  0.05
4  Athletics & Sports  0.05


----Beckton----
                    venue  freq
0  Furniture / Home Store  0.12
1             Supermarket  0.12
2      Light Rail Station  0.08
3             Coffee Shop  0.08
4           Grocery Store  0.08


----Bedford Park----
                venue  freq
0         Coffee Shop  0.09
1                 Pub  0.09
2                Café  0.07
3              Bakery  0.06
4  Italian Restaurant  0.04


----Bethnal Green----
     

           venue  freq
0            Pub  0.11
1  Grocery Store  0.06
2          Hotel  0.05
3           Café  0.05
4    Pizza Place  0.04


----Mile End----
          venue  freq
0           Pub  0.19
1          Café  0.08
2  Burger Joint  0.05
3   Pizza Place  0.05
4          Park  0.05


----Millwall----
               venue  freq
0  Indian Restaurant  0.09
1              Hotel  0.06
2           Bus Stop  0.06
3               Park  0.06
4                Pub  0.05


----Neasden----
                     venue  freq
0            Grocery Store  0.10
1     Fast Food Restaurant  0.10
2           Sandwich Place  0.10
3  Scandinavian Restaurant  0.10
4    Portuguese Restaurant  0.05


----Newbury Park----
                 venue  freq
0  Sporting Goods Shop  0.22
1                Hotel  0.11
2           Playground  0.11
3    Indian Restaurant  0.11
4          Social Club  0.11


----North Woolwich----
                  venue  freq
0        Clothing Store  0.08
1                 Hotel  0.05
2 

In [106]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



In [107]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aldborough Hatch,Metro Station,Sporting Goods Shop,Café,Supermarket,Steakhouse,Indian Restaurant,Social Club,Soccer Field,Flea Market,Fish Market
1,Alperton,Indian Restaurant,Clothing Store,Supermarket,Soccer Field,Bus Stop,Sandwich Place,Electronics Store,Fast Food Restaurant,Café,Pub
2,Barkingside,Supermarket,Indian Restaurant,Café,Middle Eastern Restaurant,Steakhouse,Sandwich Place,Sporting Goods Shop,Coffee Shop,Pub,Greek Restaurant
3,Beckton,Supermarket,Furniture / Home Store,Light Rail Station,Coffee Shop,Grocery Store,Shopping Plaza,Soccer Field,Clothing Store,Café,Bus Stop
4,Bedford Park,Pub,Coffee Shop,Café,Bakery,Park,Italian Restaurant,Burger Joint,Grocery Store,Ice Cream Shop,French Restaurant


### Clustering Neighborhoods

In [109]:
from scipy.spatial.distance import cdist

# k means determine k
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(london_grouped_clustering)
    kmeanModel.fit(london_grouped_clustering)
    distortions.append(sum(np.min(cdist(london_grouped_clustering, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / london_grouped_clustering.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

NameError: name 'london_grouped_clustering' is not defined

In [110]:
# set number of clusters
kclusters = 4

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 1, 2, 2, 1, 3, 1, 2], dtype=int32)

In [111]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

london_merged.head() # check the last columns!

,Neighborhood,Borough,Post_town,Post_code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aldborough Hatch,Redbridge,ILFORD,IG2,51.585590,0.098750,0,Metro Station,Sporting Goods Shop,Café,Supermarket,Steakhouse,Indian Restaurant,Social Club,Soccer Field,Flea Market,Fish Market
1,Alperton,Brent,WEMBLEY,HA0,51.540804,-0.300096,0,Indian Restaurant,Clothing Store,Supermarket,Soccer Field,Bus Stop,Sandwich Place,Electronics Store,Fast Food Restaurant,Café,Pub
2,Barkingside,Redbridge,ILFORD,IG6,51.585818,0.088624,0,Supermarket,Indian Restaurant,Café,Middle Eastern Restaurant,Steakhouse,Sandwich Place,Sporting Goods Shop,Coffee Shop,Pub,Greek Restaurant
3,Beckton,Newham,"LONDON, BARKING","E6, E16, IG11",51.516080,0.059426,1,Supermarket,Furniture / Home Store,Light Rail Station,Coffee Shop,Grocery Store,Shopping Plaza,Soccer Field,Clothing Store,Café,Bus Stop
4,Bedford Park,Ealing,LONDON,W4,51.498020,-0.255647,2,Pub,Coffee Shop,Café,Bakery,Park,Italian Restaurant,Burger Joint,Grocery Store,Ice Cream Shop,French Restaurant


In [112]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Neighborhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examning the clusters

In [113]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Redbridge,0.098750,0,Metro Station,Sporting Goods Shop,Café,Supermarket,Steakhouse,Indian Restaurant,Social Club,Soccer Field,Flea Market,Fish Market
1,Brent,-0.300096,0,Indian Restaurant,Clothing Store,Supermarket,Soccer Field,Bus Stop,Sandwich Place,Electronics Store,Fast Food Restaurant,Café,Pub
2,Redbridge,0.088624,0,Supermarket,Indian Restaurant,Café,Middle Eastern Restaurant,Steakhouse,Sandwich Place,Sporting Goods Shop,Coffee Shop,Pub,Greek Restaurant
14,Waltham Forest,0.015682,0,Restaurant,Grocery Store,Playground,Soccer Field,Garden Center,Beer Garden,Bar,Thai Restaurant,Italian Restaurant,Pub
17,Brent,-0.191890,0,Turkish Restaurant,Supermarket,Indian Restaurant,Pizza Place,Coffee Shop,Pub,Restaurant,Japanese Restaurant,Park,Gym / Fitness Center
22,Brent,-0.239021,0,Park,Yoga Studio,Italian Restaurant,Pizza Place,Discount Store,Coffee Shop,Restaurant,Sandwich Place,Fast Food Restaurant,Café
25,Newham,0.055320,0,Pub,Fast Food Restaurant,Indian Restaurant,Grocery Store,Supermarket,Park,Sandwich Place,Discount Store,Furniture / Home Store,Electronics Store
26,Hillingdon,-0.401653,0,Grocery Store,Metro Station,Indian Restaurant,Sandwich Place,Bakery,Coffee Shop,Café,Supermarket,Bus Stop,Park
27,Hounslow,-0.410659,0,Pharmacy,Supermarket,Clothing Store,Historic Site,Portuguese Restaurant,Fast Food Restaurant,Movie Theater,Sandwich Place,Bar,Park
29,Redbridge,0.065308,0,Pizza Place,Grocery Store,Thai Restaurant,Park,Pub,Chinese Restaurant,Nightclub,Museum,Café,Soccer Field


In [114]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Newham,0.059426,1,Supermarket,Furniture / Home Store,Light Rail Station,Coffee Shop,Grocery Store,Shopping Plaza,Soccer Field,Clothing Store,Café,Bus Stop
6,Tower Hamlets,-0.007184,1,Coffee Shop,Sushi Restaurant,Italian Restaurant,Park,Hotel,Burger Joint,Sandwich Place,Hotel Bar,Café,Juice Bar
9,Brent,-0.275760,1,Coffee Shop,Bar,Clothing Store,Hotel,Sandwich Place,Supermarket,Sporting Goods Shop,Warehouse Store,Indian Restaurant,Food Court
13,Tower Hamlets,-0.025717,1,Park,Coffee Shop,Hotel,Plaza,Gym / Fitness Center,Italian Restaurant,Burger Joint,Café,Indian Restaurant,Hotel Bar
15,Newham,0.008299,1,Grocery Store,Café,Hotel,Gym / Fitness Center,Coffee Shop,Fast Food Restaurant,Platform,Park,Science Museum,Sandwich Place
19,Hounslow,-0.409677,1,Bus Stop,Hotel,Restaurant,Pizza Place,Park,Garden Center,Smoke Shop,Fast Food Restaurant,Deli / Bodega,Middle Eastern Restaurant
20,Tower Hamlets,-0.008243,1,Indian Restaurant,Pub,Bus Stop,Park,Business Service,Sandwich Place,Hotel,Bubble Tea Shop,Café,River
21,Newham,-0.082644,1,Hotel,Coffee Shop,Gym / Fitness Center,Restaurant,Hotel Bar,Garden,Scenic Lookout,Asian Restaurant,Italian Restaurant,Steakhouse
39,Hillingdon,-0.436303,1,Hotel,Bus Stop,Hotel Bar,Pub,Indian Restaurant,Gym,Coffee Shop,Rental Car Location,Restaurant,Pool
40,Hillingdon,-0.475544,1,Coffee Shop,Hotel,Restaurant,Bar,Bus Stop,Gym / Fitness Center,Bed & Breakfast,Rental Car Location,Fast Food Restaurant,Chinese Restaurant


In [115]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Ealing,-0.255647,2,Pub,Coffee Shop,Café,Bakery,Park,Italian Restaurant,Burger Joint,Grocery Store,Ice Cream Shop,French Restaurant
5,Tower Hamlets,-0.056163,2,Coffee Shop,Pub,Café,Cocktail Bar,Hotel,Restaurant,Bookstore,Flower Shop,Bakery,Convenience Store
8,Hounslow,-0.321662,2,Coffee Shop,Pub,Café,Hotel,Gym,Italian Restaurant,Gastropub,Sandwich Place,Canal Lock,Asian Restaurant
10,Tower Hamlets,0.014814,2,Clothing Store,Pub,Coffee Shop,Café,Bar,Indian Restaurant,Supermarket,Burger Joint,Park,Gym / Fitness Center
11,Brent,-0.202596,2,Pub,Coffee Shop,Farmers Market,Bakery,Italian Restaurant,Brazilian Restaurant,Middle Eastern Restaurant,Pizza Place,Mediterranean Restaurant,Cocktail Bar
12,Tower Hamlets,-0.057422,2,Coffee Shop,Pub,Café,Cocktail Bar,Hotel,Bakery,Bookstore,Wine Bar,Flower Shop,Restaurant
18,Hillingdon,-0.476384,2,Pub,Restaurant,Coffee Shop,Hotel,Sandwich Place,Canal Lock,Supermarket,Café,Nightclub,Gym
23,Ealing,-0.305195,2,Coffee Shop,Pub,Hotel,Park,Café,Thai Restaurant,Italian Restaurant,Pizza Place,Burger Joint,Bakery
24,Hounslow,-0.441359,2,Coffee Shop,Café,Burger Joint,IT Services,Indian Restaurant,Restaurant,Pub,Grocery Store,Fast Food Restaurant,Yoga Studio
31,Ealing,-0.345351,2,Clothing Store,Pub,Hotel,Bakery,Portuguese Restaurant,Coffee Shop,Chinese Restaurant,Sandwich Place,Café,Fast Food Restaurant


In [116]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Tower Hamlets,-0.019482,3,Pub,Grocery Store,Café,Coffee Shop,Bar,Hotel,Pharmacy,Bike Rental / Bike Share,Burger Joint,Bus Stop
16,Waltham Forest,0.003996,3,Pub,Italian Restaurant,Coffee Shop,Fast Food Restaurant,History Museum,Nature Preserve,Convenience Store,Bakery,Park,Grocery Store
28,Newham,0.024925,3,Grocery Store,Hotel,Bus Stop,Pub,Café,Market,Playground,Bar,Ice Cream Shop,Comfort Food Restaurant
37,Hillingdon,-0.482562,3,Café,Canal Lock,Grocery Store,Gastropub,Pub,Sports Club,Yoga Studio,Farmers Market,Factory,Falafel Restaurant
45,Hillingdon,-0.448335,3,Chinese Restaurant,Fast Food Restaurant,Pub,Grocery Store,Park,Pharmacy,Plaza,Construction & Landscaping,Farmers Market,Ethiopian Restaurant
47,Hillingdon,-0.442351,3,Pub,Grocery Store,Chinese Restaurant,Train Station,Bakery,Park,Bus Stop,Indian Restaurant,Fish & Chips Shop,Metro Station
50,Hounslow,-0.326311,3,Pub,Bus Stop,Grocery Store,Pharmacy,Fast Food Restaurant,River,Restaurant,Coffee Shop,Italian Restaurant,Trail
56,Waltham Forest,0.006424,3,Pub,Grocery Store,Café,Fast Food Restaurant,Coffee Shop,Pizza Place,Hotel,Pharmacy,Stationery Store,Burger Joint
60,Redbridge,0.084051,3,Pizza Place,Park,Betting Shop,Portuguese Restaurant,Gas Station,Ice Cream Shop,Pub,Grocery Store,Exhibit,Factory
70,Ealing,-0.369377,3,Canal Lock,Ice Cream Shop,Grocery Store,Pub,Gym / Fitness Center,Yoga Studio,Fast Food Restaurant,Exhibit,Factory,Falafel Restaurant
